In [50]:

var fs = require("fs")
var tf = require("../../jslib/textFile")
var funcTools = require("../../jslib/funcTools")
var palette = require("../../jslib/palette.js")

var ls = directoryPath => (filterFileFunc = path => file => fs.statSync(path+file).isFile()) => {
    const files = fs.readdirSync(directoryPath,(err)=>{
        if (err) throw err;   
    })
    return {
        directory : directoryPath,
        files : files.filter(filterFileFunc(directoryPath))
    }
} 

var fetchData = (rp,dp)=>{
    const metaFiles = ls(rp)(path => file =>  fs.statSync(path+file).isFile() && /^meta.*\.json$/.test(file))
    const sampleFiles = ls(rp)(path => file =>  fs.statSync(path+file).isFile() && /.*\.csv$/.test(file))
    const summaryFiles = ls(rp)(path => file => file === "summary.csv");
    console.log(metaFiles)
    console.log(sampleFiles)

    const meta = JSON.parse(fs.readFileSync(`${metaFiles.directory}${metaFiles.files.pop()}`))
    console.log(meta)

    const data = tf.text2Dataframe(fs.readFileSync(`${dp}${meta.data_file}`,"utf-8"),"csv")
    const error = tf.text2Dataframe(fs.readFileSync(`${dp}${meta.data_file}`,"utf-8"),"csv")
    const option = JSON.parse(fs.readFileSync("../model/"+meta.option_file,"utf-8"))


    console.log(option)

    const summary = tf.text2Dataframe(fs.readFileSync(`${summaryFiles.directory}${summaryFiles.files[0]}`,"utf-8"),"csv")
    console.log(Object.keys(df_summarized))
    return {
        meta,
        data,
        error,
        option,
        summary
    }
}

var getSummarizedParameters = (df, by, initial_parameters) => {
    const ps = [...initial_parameters];
    const splited = df.parameter.map((k,i)=>{
        let parameter = k.slice(0,k.length-1)
        let num = k.slice(k.length-1)
        return [parameter,num]
    });
    
    splited.map(([parameter,num],i)=>{
        if (0 < i && i < splited.length -1){
            ps[num][parameter] = df[by][i]
        }
    })
    return ps
}

var groupEachStage = (parameters) => {
    const obj = {};
    parameters.map((p,i)=>{
        if (i === 0){
            Object.keys(p).map(k=>{
              obj[k] = []  
            })
        }
        Object.entries(p).map(([k,v])=>{
            obj[k][i] = v
        })
    })
    return obj
}

var Plotly = require("ijavascript-plotly");

In [46]:
var integratedLiquidLine = (magma,cmap) => {
    const  differentiationLines = magma.custom.differentiationLineStack;
    const  mixingLines = magma.custom.mixingLineStack;
    

    return (x,y) => funcTools.zip(mixingLines)(differentiationLines).map(
            ([m,d],i) => {
                if (i === 0){
                    return [
                    {
                        x : [m[x][0]],
                        y : [m[y][0]],
                        name : "Current WR",
                        mode : "markers",
                        marker : {
                            size : 8,
                            color : "#000000"
                        }
                    },
                    {
                        x : d[x],
                        y : d[y],
                        mode : "lines",
                        line : {
                            width : 2,
                            color : cmap[i]
                        }
                    },
                    {
                            x : m[x],
                            y : m[y],
                            mode : "markers",
                            marker : {
                                size : 4,
                                color : cmap[i]
                            }
                    }
                    ]
                }else{
                    return [
                        {
                            x : m[x],
                            y : m[y],
                            mode : "markers",
                            marker : {
                                size : 4,
                                color : cmap[i]
                            }
                        },
                        {
                            x : d[x],
                            y : d[y],
                            mode : "lines",
                            line : {
                                width : 2,
                                color : cmap[i]
                            }
                        }
                    ]
                }
            }
        ).reduce((a,b)=>[...a,...b],[])
        
}


Directory

```
/diffusion
/phase
/ptmcmc
    /model
    /tool
    /result
    
```

In [87]:
var resultPath = "../result/1002-9-1C-opx1/2018-0714/"
var dataPath = "../model/"

In [88]:
/**
    Not modify
*/
var {
    meta,
    data,
    error,
    option,
    summary
} = fetchData(resultPath,dataPath)


var {
      model,
      parameters,
      updateCondition,
      constrain,
      magma
    } = require("../"+meta.model)(option)

var cmap = palette("tol-rainbow", parameters.length)

var summarizedParameters = getSummarizedParameters(summary, "mode", parameters);

var parametersEachStage = groupEachStage(summarizedParameters)

summarizedParameters[0]

{ directory: '../result/1002-9-1C-opx1/2018-0714/',
  files:
   [ 'meta-2018_0714_141849.json', 'meta-2018_0714_190558.json' ] }
{ directory: '../result/1002-9-1C-opx1/2018-0714/',
  files:
   [ 'sample-0-2018_0714_141849.csv',
     'sample-0-2018_0714_190558.csv',
     'sample-1-2018_0714_141849.csv',
     'sample-1-2018_0714_190558.csv',
     'sample-10-2018_0714_141849.csv',
     'sample-10-2018_0714_190558.csv',
     'sample-11-2018_0714_141849.csv',
     'sample-11-2018_0714_190558.csv',
     'sample-12-2018_0714_141849.csv',
     'sample-12-2018_0714_190558.csv',
     'sample-13-2018_0714_141849.csv',
     'sample-13-2018_0714_190558.csv',
     'sample-14-2018_0714_141849.csv',
     'sample-14-2018_0714_190558.csv',
     'sample-15-2018_0714_141849.csv',
     'sample-15-2018_0714_190558.csv',
     'sample-2-2018_0714_141849.csv',
     'sample-2-2018_0714_190558.csv',
     'sample-3-2018_0714_141849.csv',
     'sample-3-2018_0714_190558.csv',
     'sample-4-2018_0714_141849.csv',


Error: ENOENT: no such file or directory, open '../result/1002-9-1C-opx1/2018-0714/undefined'

In [79]:
Plotly([
    {
        y : parametersEachStage.growth_stoichiometry_orthopyroxene,
        name : "Orthopyroxene"
    },
    {
        y : parametersEachStage.growth_stoichiometry_spinel,
        name : "Spinel"
    }
],{
    title : "Stoichiometry in crystal growth",
    xaxis : {
        title : "Stage number"
    },
    yaxis : {
        title : "Stoichiometry",
        range : [0,1]
    }
})

In [80]:
Plotly([
    {
        y : parametersEachStage.log10_tau
    }
],{
    
    title : "Logarithm of net diffusion scale",
    xaxis : {
        title : "Stage number"
    },
    yaxis : {
        title : "log110 tau"
    }

})

In [81]:
var modeled = model(summarizedParameters,data)
var ndProfiles = magma.diffusionProfiles.orthopyroxene;

var ndFe_Mg = ndProfiles.Fe_Mg.notDiffusedProfile.get()

Plotly([
    {
        x : modeled.x,
        y : modeled.Fe_Mg.map(v=>100/(1+v)),
        name : "Model: diffused"
    },
     {
        x : ndFe_Mg.x,
        y : ndFe_Mg.c.map(v=>100/(1+v)),
        name : "Model: original"
    },
    {
        x : data.x,
        y : data.Fe_Mg.map(v=>100/(1+parseFloat(v))),
        mode : "markers",
        name : "Observed"
    }

],{
    width : 800,
    height : 600,
    xaxis : {
        title : "Radius"
    },
    yaxis : {
        title : "Mg#"
    }
})

In [82]:
var modeled = model(summarizedParameters,data)
var ndProfiles = magma.diffusionProfiles.orthopyroxene;

var ndFe_Mg = ndProfiles.Fe_Mg.notDiffusedProfile.get()

Plotly([
    {
        x : modeled.x,
        y : modeled.Fe_Mg,
        name : "Model: diffused"
    },
     {
        x : ndFe_Mg.x,
        y : ndFe_Mg.c,
        name : "Model: original"
    },
    {
        x : data.x,
        y : data.Fe_Mg,
        mode : "markers",
        name : "Observed"
    }

],{
    width : 800,
    height : 600,
    xaxis : {
        title : "Radius"
    },
    yaxis : {
        title : "Fe/Mg"
    }
})

In [83]:
var ndCr2O3 = ndProfiles.Cr2O3.notDiffusedProfile.get()

Plotly([
    {
        x : modeled.x,
        y : modeled.Cr2O3,
        name : "Model: diffused"
    },
    {
        x : ndCr2O3.x,
        y : ndCr2O3.c,
        name : "Model: original"
    },
    {
        x : data.x,
        y : data.Cr2O3,
        mode : "markers",
        name : "Observed"
    }

],{
    width : 800,
    height : 600,
    xaxis : {
        title : "Radius"
    },
    yaxis : {
        title : "Cr2O3"
    }
})

In [84]:

var liquidLine = integratedLiquidLine(magma,cmap)

//console.log(liquidLine("SiO2","MgO"))


In [85]:

Plotly(liquidLine("SiO2","MgO"),{
    width : 1000,
    height : 1000,
    xaxis : {
        title :"SiO2"
    },
    yaxis : {
        title : "MgO"
    }
})


In [86]:
Plotly(liquidLine("SiO2","Cr2O3"),{
    width : 1000,
    height : 1000,
    xaxis : {
        title :"SiO2"
    },
    yaxis : {
        title : "Cr2O3"
    }
})
